In [ ]:
import re
import os

link = r'\[[^@\\!].+?\]\(.+?\)'
inpage_link=r'\[[^@\\!][^\[]+?\]\(#.+?\)'
herf = r'\(.+?\)'
title =r'\n#+.+'
herf_dic={}

In [ ]:
def get_herf_dic(matched,get_old):
    '''获取新旧链接映射表,并按请求(get_old)返回对应链接'''
    global herf_dic,index
    new,old = re.sub('\[|\)','',matched).split('](')
    chapter=int(re.findall('\d+',new)[1])+index
    
    # Herf transformation rules
    new=re.sub('\.','',new)
    new=re.sub('\\\\ | ','-',new).lower().replace('(','（').replace(')','）')
    new='/docs/{}?id=_'.format(chapter)+new
    
    if old not in herf_dic.keys():
        herf_dic[old]=new
    
    if get_old:
        return old
    else:
        return new

In [ ]:
for index in [1,19,33]:
    with open('./docs/{}.md'.format(index), 'r+') as f:
        text = f.read()
    links=re.findall(link,text)
    for l in links:
        get_herf_dic(l,False)

In [ ]:
with open('./docs/herf_dic.json','w') as f:
        f.write(str(herf_dic))
        

In [ ]:
def find_in_dic(link):
    '''按照新旧链接映射表,完成链接修改'''
    global herf_dic
    link=link.group()
    name=link.split('](')[0].replace('[','')
    old_herf= get_herf_dic(link,True)
    if old_herf in herf_dic.keys():
        new_link = "[{}]({})".format(name,herf_dic[old_herf])
        return new_link
    else:
        return link

In [ ]:
for index in [1,19,33]:
    with open('./docs/{}.md'.format(index), 'r+') as f:
        text = f.read()
        text = re.sub(link,find_in_dic,text)
        f.seek(0)
        f.truncate()
        f.write(text)

In [ ]:
# 寻找页内链接
with open('./docs/2.md', 'r+') as f:
    text = f.read()
herf_list = re.findall(inpage_link,text)
titles = re.findall(title,text)
for t in titles:
    t= re.sub('\.|\n','',t)
    t= re.sub('#+','#',t)
    t = re.sub('\\\\ | ','-',t).lower().replace('(','（').replace(')','）')
    print(t)